# Notebook configuration

In [3]:
import os
import json
import re
import requests
import datetime
from typing import List, Optional, Any, Dict
from urllib.parse import urljoin
import ollama

import pandas as pd
import magic
from bs4 import BeautifulSoup
from markdownify import markdownify
from requests.exceptions import RequestException

from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import (
    PyMuPDFLoader, CSVLoader, JSONLoader, UnstructuredXMLLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from langchain_ollama import OllamaLLM, OllamaEmbeddings

from smolagents import (
    CodeAgent, 
    LiteLLMModel, 
    DuckDuckGoSearchTool, 
    ToolCallingAgent, 
    tool, 
    VisitWebpageTool,
    GoogleSearchTool
)

from abc import abstractmethod, ABC
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

#############################################################

oc = ollama.Client("http://localhost:11434")

Data_dir = "/users/formation/irtn7prtnc/LLM_Valdom/Dataset"
Download_dir = "/users/formation/irtn7prtnc/LLM_Valdom/Download"

os.makedirs(Data_dir, exist_ok = True)
os.makedirs(Download_dir, exist_ok = True)

model = LiteLLMModel(
    model_id = "ollama/qwen2.5-coder:32b", #['deepseek-r1:32b', 'qwen2.5-coder:32b', 'llama3.1:8b', 'mistral-nemo:latest', 'mistral:latest']
    api_base = "http://localhost:11434/api/generate",
    num_ctx = 24000
    )

# Mise en place du RAG

### RAG definition (Local)

In [52]:
class RAGInterface(ABC):
    """
    Abstract class defining a generic RAG system. 
    
    This class ensures that all RAG implementations follow a common structure.
    """
    def __init__(self, name: str, knowledge_db: Optional[Any] = None):
        self.name = name  # Identifier for the RAG system
        self.knowledge_db = knowledge_db  # Storage backend (e.g., a vector database)
    
    @abstractmethod
    def retrieve(self, query: str) -> List[Document]:
        """
        Retrieve relevant contexts from the knowledge_db based on the query.
        Args:
            query (str): The user query.
        Returns:
            List[Document]: Retrieved document chunks.
        """
        pass
    
    @abstractmethod
    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """
        Generate a response based on the query and retrieved contexts.
        Args:
            query (str): The user query.
            retrieved_contexts (List[Document]): Relevant document chunks.
        Returns:
            str: The generated response.
        """
        pass

# Default prompt template for RAG
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use four sentences maximum and keep the answer concise.

Question: {query}
Context: {retrieved_contexts}
Answer:
"""
######################## RAG BM25 ########################

class BM25V0RAG(RAGInterface):
    """
    Sparse Retrieval RAG using BM25 without embeddings for generation.
    
    - Stores text chunks in Qdrant using BM25 sparse retrieval.
    - Retrieves the top-k relevant chunks based on keyword matching.
    - Uses a language model to generate answers from retrieved contexts.
    """

    def __init__(self, generation_model: OllamaLLM, docs_v0: List[Document]):
        # Initialize BM25 sparse retrieval (no embeddings for generation)
        sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

        # Store documents in Qdrant using sparse retrieval (BM25)
        self.knowledge_db = QdrantVectorStore.from_documents(
            docs_v0,  
            embedding = OllamaEmbeddings(model="mistral"),  # No embeddings used in this mode
            sparse_embedding =sparse_embeddings,  # BM25 sparse embeddings
            location = ":memory:",  # Store in-memory (can be changed to persistent storage)
            collection_name = "rag_bm",  # Collection name for BM25-based retrieval
            retrieval_mode = RetrievalMode.SPARSE,  # Use only sparse retrieval (BM25)
        )

        # Define model name dynamically
        name = f"bm25_v0_{generation_model.model}"
        super().__init__(name = name, knowledge_db = self.knowledge_db)

        # Initialize the LLM and retriever
        self.llm = generation_model
        self.retriever = self.knowledge_db.as_retriever(
            search_type="similarity", search_kwargs={"k": 5}  # Retrieve top 5 matches
        )
        self.gen_prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)  # Use the structured prompt (no embeddings)

    def add_documents(self, new_docs: List[Document]):
        """
        Add new documents to Qdrant Cloud using BM25 sparse retrieval without embeddings.
        This method adds documents directly using keyword matching (BM25).
        """

        # Here we assume that new_docs are pre-processed and are in a list of Document objects
        for doc in new_docs:
            try:
                self.knowledge_db.add_documents([doc]) 
            except Exception as e:
                print(f"[Warning] Erreur lors de l'ajout du document {doc.metadata.get('source', 'unknown')}: {e}")

    def retrieve(self, query: str) -> List[Document]:
        """Retrieve relevant documents for a given query."""
        return self.retriever.invoke(query)

    def find_relevant_documents(self, query: str) -> List[str]:
        """Find sources of relevant documents."""
        retrieved = self.retrieve(query)
        return list(set(doc.metadata.get("source", "unknown") for doc in retrieved))

    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """
        Generates a response using the retrieved contexts.

        Args:
            query (str): The user query.
            retrieved_contexts (List[Document]): Retrieved document chunks based on BM25.

        Returns:
            str: The generated answer from the language model.
        """
        # Format retrieved contexts into a single string
        format_retrieved_contexts = "\n".join([rc.page_content for rc in retrieved_contexts])

        # Format the query with the retrieved contexts for generation
        augmented_query = self.gen_prompt.format(
            query=query,
            retrieved_contexts=format_retrieved_contexts
        )

        # Generate the final response
        response = self.llm.invoke(augmented_query)
        return response

######################## RAG Hybride ########################

class HybridRAG(RAGInterface):
    """
    Retrieval-Augmented Generation with hybrid search (dense + sparse).
    """
    def __init__(
        self, 
        generation_model: OllamaLLM, 
        docs: List[Document], 
        collection_name: str = "rag_hybrid", 
        alpha: float = 0.7,
        embedding_model: str = "mistral:latest"):
        
        # Configurable sparse and dense embeddings
        sparse_embeddings = FastEmbedSparse(model_name = "Qdrant/bm25", cache_dir=".")
        dense_embeddings = OllamaEmbeddings(model = embedding_model)
        
        # Initialize vector store with hybrid search
        self.vectorstore = QdrantVectorStore.from_documents(
            docs,
            embedding = dense_embeddings,
            sparse_embedding = sparse_embeddings,
            location = ":memory:",
            collection_name = collection_name,
            retrieval_mode = RetrievalMode.HYBRID,
            sparse_dense_ratio = alpha  # 0 = pure dense, 1 = pure sparse
        )
        
        # Name with generation model
        name = f"hybrid_{generation_model.model}"
        super().__init__(name = name, knowledge_db = self.vectorstore)
        
        self.llm = generation_model
        self.retriever = self.vectorstore.as_retriever(search_kwargs = {"k": 20})
        self.gen_prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
    
    def add_documents(self, new_docs: List[Document]):
        """Add new documents to the vector store."""
        self.vectorstore.add_documents(new_docs)
    
    def retrieve(self, query: str) -> List[Document]:
        """Retrieve relevant documents for a given query."""
        return self.retriever.invoke(query)
    
    def generate(self, query: str, retrieved_contexts: List[Document]) -> str:
        """Generate a response based on retrieved contexts."""
        context_str = "\n".join([doc.page_content for doc in retrieved_contexts])
        full_prompt = self.gen_prompt.format(query=query, retrieved_contexts=context_str)
        return self.llm.invoke(full_prompt)
    
    def find_relevant_documents(self, query: str) -> List[str]:
        """Find sources of relevant documents."""
        retrieved = self.retrieve(query)
        return list(set(doc.metadata.get("source", "unknown") for doc in retrieved))

### File pre-processing & RAG creation

In [57]:
def load_and_chunk_file(file_path: str) -> List[Document]:
    ext = file_path.split('.')[-1].lower()

    if ext == "pdf":
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()
        for d in docs:
            d.page_content = " ".join(d.page_content.split())
        splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
        return splitter.split_documents(docs)

    elif ext == "csv":
        df = pd.read_csv(file_path)
        if len(df) > 10000:
            df = df.head(10000)
        docs = [
            Document(
                page_content=" ".join(str(value) for value in row if pd.notna(value)), 
                metadata={"source": file_path, "row_index": index}
            )
            for index, row in df.iterrows()
        ]
        splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
        return splitter.split_documents(docs)

    elif ext == "json":
        try:
            from langchain_community.document_loaders import JSONLoader
            loader = JSONLoader(file_path, jq_schema=".", text_content=False)
            docs = loader.load()
        except ImportError:
            print(f"[Warning] jq non installé. Chargement brut de {file_path}")
            with open(file_path, "r", encoding="utf-8") as f:
                raw = f.read()
            docs = [Document(page_content=raw, metadata={"source": file_path})]
        
        for d in docs:
            try:
                data = json.loads(d.page_content)
                d.page_content = json.dumps(data, indent=2)
            except:
                pass
        splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
        return splitter.split_documents(docs)

    elif ext == "xml":
        loader = UnstructuredXMLLoader(file_path)
        docs = loader.load()
        for d in docs:
            d.page_content = " ".join(d.page_content.split())
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        return splitter.split_documents(docs)

    else:
        raise ValueError(f"Extension non supportée : {ext}")

#############################################################

def get_all_files_recursively(folder_path: str, valid_extensions: Optional[List[str]] = None) -> List[str]:
    """
    Parcourt récursivement un dossier et retourne tous les fichiers avec les extensions valides.
    """
    all_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            ext = file.lower().split('.')[-1]
            if valid_extensions is None or ext in valid_extensions:
                full_path = os.path.join(root, file)
    return all_files

#############################################################

def build_hybrid_rag_from_folder(folder_path: str, alpha: float = 0.7) -> HybridRAG:
    """
    Construit un système HybridRAG à partir des fichiers dans un dossier (récursivement).
    """
    valid_extensions = ["pdf", "csv", "json", "xml"]
    file_paths = get_all_files_recursively(folder_path, valid_extensions)

    all_docs = []
    for path in file_paths:
        try:
            docs = load_and_chunk_file(path)
            print(f"Documents chargés pour {path}: {len(docs)} documents")
            all_docs.extend(docs)
        except Exception as e:
            print(f"[Warning] Erreur lors du traitement de {path} : {e}")

    llm = OllamaLLM(model="mistral:latest")
    rag = HybridRAG(generation_model=llm, docs=all_docs, alpha = alpha)
    return rag

#############################################################

def build_rag_from_folder(folder_path: str) -> BM25V0RAG:
    """
    Construit un système RAG à partir des fichiers dans un dossier (récursivement).
    """
    valid_extensions = ["pdf", "csv", "json", "xml"]
    file_paths = get_all_files_recursively(folder_path, valid_extensions)

    all_docs = []
    for path in file_paths:
        try:
            docs = load_and_chunk_file(path)
            print(f"Documents chargés pour {path}: {len(docs)} documents")
            all_docs.extend(docs)
        except Exception as e:
            print(f"[Warning] Erreur lors du traitement de {path} : {e}")

    llm = OllamaLLM(model="mistral:latest")
    # Remplacer "docs" par "docs_v0" pour correspondre au constructeur
    rag = BM25V0RAG(generation_model=llm, docs_v0=all_docs)
    return rag

### Création du RAG

In [58]:
rag = build_rag_from_folder("/users/formation/irtn7prtnc/llm_engineering/Data") 

2025-03-27 13:01:29.430 | ERROR    | fastembed.common.model_management:download_model:429 - Could not download model from HuggingFace: [Errno 13] Permission denied: '/tmp/fastembed_cache/models--Qdrant--bm25' Falling back to other sources.
2025-03-27 13:01:29.433 | ERROR    | fastembed.common.model_management:download_model:450 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.
2025-03-27 13:01:32.669 | ERROR    | fastembed.common.model_management:download_model:429 - Could not download model from HuggingFace: [Errno 13] Permission denied: '/tmp/fastembed_cache/models--Qdrant--bm25' Falling back to other sources.
2025-03-27 13:01:32.672 | ERROR    | fastembed.common.model_management:download_model:450 - Could not download model from either source, sleeping for 9.0 seconds, 1 retries left.
2025-03-27 13:01:41.900 | ERROR    | fastembed.common.model_management:download_model:429 - Could not download model from HuggingFace: [Errno 13] Permission denie

ValueError: Could not load model Qdrant/bm25 from any source.

In [49]:
 # Agent pourrait appeler ceci :
query = "Quelle est le député le plus jeunes ?"
sources = rag.find_relevant_documents(query)
print("🔍 Documents pertinents :", sources)

answer = rag.generate(query, rag.retrieve(query))
print("\n🧠 Réponse :", answer)

🔍 Documents pertinents : []

🧠 Réponse :  Le député le plus jeune actuellement en France est Fabien Riu, né le 17 novembre 1982. Il représente la Seine-Saint-Denis depuis les élections législatives de juin 2017.


# Mise en place des Agents

### RAG Agent tools

In [12]:
@tool
def agent_add_file_to_rag(rag : str, path: str) -> str:
    """
    Loads a supported file (PDF, CSV, JSON, XML), splits it into chunks, 
    and adds them to the RAG (Retrieval-Augmented Generation) system for future retrieval.
    
    This function is capable of handling both single files and entire directories. 
    If the path is a directory, all supported files within the directory (and its subdirectories) 
    will be processed.

    Args:
        rag: The RAG system or retriever to which the chunks will be added. This is the core component that stores and retrieves documents during question-answering tasks.
        path: The path to the file or directory to be processed. Supported file formats include: pdf, csv, json & xml.

    Returns:
        str: A message indicating the number of chunks added to the RAG, or an error message if the file format is unsupported or if an error occurs during processing.
    """
    try:
        docs = []
        if os.path.isfile(path):
            if path.endswith(('.pdf', '.csv', '.json', '.xml')):
                docs = load_and_chunk_file(path)
            else:
                return f"Format de fichier non supporté : {path}"

        elif os.path.isdir(path):
            valid_exts = ('.pdf', '.csv', '.json', '.xml')
            for root, _, files in os.walk(path):
                for f in files:
                    full_path = os.path.join(root, f)
                    if full_path.endswith(valid_exts):
                        try:
                            docs.extend(load_and_chunk_file(full_path))
                        except Exception as e:
                            print(f"[Erreur] Chargement échoué pour {full_path} : {e}")
        rag.add_documents(docs)
        return f"{len(docs)} chunks ajoutés depuis {path}"
        
    except Exception as e:
        return f"Le chemin donné n'est ni un fichier ni un dossier : {path}"

######

@tool
def detect_file_type(file_path: str) -> str:
    """Detects the MIME type of a file.

    Args:
        file_path: The path of the file that we want the type.

    Returns:
        Tue file type of an error message if it's not possible to detecting the type.
    """
    try:
        mime = magic.Magic(mime=True)
        return mime.from_file(file_path)

    except Exception as e:
        return f"Error detecting file type: {str(e)}"
    
######

@tool
def extract_any_archive(file_path: str, destination: str = None) -> str:
    """Extracts a archive file to a specified directory.

    Args:
        file_path: The path of the file to extract.
        destination: The destination of the extracted file.
    Returns:
        The extracted file path, or an error message if file extraction failed.
    """
    try:
        if destination is None:
            destination = os.path.splitext(file_path)[0]

        patoolib.extract_archive(file_path, outdir=destination)

        os.remove(zip_path)

        return f"Archive extracted successfully to: {destination}"

    except Exception as e:
        return f"Error extracting archive: {str(e)}"
    
######

@tool
def move_file(source: str, destination: str) -> str:
    """Moves a file or directory to a new location.

    Args:
        source: The current path of the file to move.
        destination: The new path of the file.

    Returns:
        The new file path, or an error message if file transfer failed
    """
    try:
        if not os.path.exists(destination):
            os.makedirs(destination)

        shutil.move(source, destination)

        return f"File successfully move to : {destination}"

    except Exception as e:
        return f"Fail during file transfert : {str(e)}"   

######

@tool
def normalize_and_ensure_unique_filename(path: str) -> str:
    """
    Normalizes the name of a file or directory and ensures that it is unique 
    by appending a numerical suffix if a file with the same name already exists.

    The function first replaces any non-alphanumeric characters (except for underscores, 
    hyphens, and periods) with underscores, and then checks if the normalized name 
    already exists. If it does, it appends a numerical suffix to the name.

    Args:
        path: The full path of the file or directory whose name is to be normalized and checked for uniqueness.

    Returns:
        str: The normalized and unique file or directory path.
    """
    base = os.path.basename(path)
    normalized_name = re.sub(r"[^a-zA-Z0-9_\-\.]", "_", base)

    # Get the directory and ensure the file name is unique
    directory = os.path.dirname(path)
    unique_path = os.path.join(directory, normalized_name)

    # Ensure uniqueness
    if not os.path.exists(unique_path):
        return unique_path

    # If the file exists, append a numerical suffix
    base, ext = os.path.splitext(normalized_name)
    i = 1
    while os.path.exists(os.path.join(directory, f"{base}_{i}{ext}")):
        i += 1

    return os.path.join(directory, f"{base}_{i}{ext}")


### Web Agent tools

In [ ]:
@tool
def visit_webpage(url: str) -> str:
    """Fetches the content of a webpage and returns it in a clean Markdown format.

    This tool sends an HTTP GET request to the provided URL, retrieves the HTML content,
    converts it into Markdown to preserve readability while removing HTML-specific elements,
    and returns the cleaned content. It automatically handles request errors and unexpected failures.

    Args:
        url: The URL of the webpage to retrieve and convert.

    Returns:
        A string containing the converted Markdown content of the webpage,
        or an error message if the request or conversion fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"

    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

##########

@tool
def summarize_webpage(url: str) -> str:
    """Fetches the content of a webpage and summarizes it in a concise paragraph.

    This tool first downloads the webpage, converts its HTML into Markdown, then
    uses a language model to summarize the key points. Ideal for previewing large pages
    or extracting meaningful information quickly.

    Args:
        url: The URL of the webpage to summarize.

    Returns:
        A summary string of the page content, or an error message if the operation fails.
    """
    try:
        from markdownify import markdownify
        import re
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Convert HTML to Markdown
        markdown_content = markdownify(response.text).strip()
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        # Trim if too long
        if len(markdown_content) > 4000:
            markdown_content = markdown_content[:4000] + "..."

        # Call LLM (Qwen or another)
        from langchain_core.runnables import Runnable
        from langchain_core.prompts import PromptTemplate
        from langchain_core.output_parsers import StrOutputParser
        from langchain_community.llms import Ollama

        llm = Ollama(model="mistral:latest")
        prompt = PromptTemplate.from_template(
            "Summarize the following web content:\n\n{content}\n\nSummary:"
        )
        chain: Runnable = prompt | llm | StrOutputParser()

        return chain.invoke({"content": markdown_content})

    except Exception as e:
        return f"Error during summarization: {str(e)}"  

##########
    
@tool
def check_url_validity(url: str) -> bool:
    """Downloads a file from a given URL to the local cache directory.

    This tool initiates a streamed download of the file pointed to by the given URL,
    saves it to a local cache folder, and returns the file path upon success.
    It supports large files by downloading them in chunks.

    Args:
        url: The direct URL of the file to download.

    Returns:
        A confirmation message with the path to the downloaded file,
        or an error message if the download fails.
    """
    
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        return response.status_code == 200

    except requests.RequestException:
        return False
    
##########

@tool
def download_file(url: str) -> str:
    """Checks if a URL is reachable and returns a boolean response.

    This tool sends an HTTP HEAD request to the target URL to verify its availability,
    following redirects if necessary. It's useful to ensure a link is valid before fetching or downloading.

    Args:
        url: The URL to verify for availability and accessibility.

    Returns:
        True if the URL is reachable (HTTP status 200), otherwise False.
    """
    try:
        local_filename = os.path.join(Download_dir, url.split('/')[-1])

        with requests.get(url, stream=True) as r:
            r.raise_for_status()

            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return f"File downloaded with succes : {local_filename}"

    except Exception as e:
        return f"Error during file downloading: {str(e)}"

##########

@tool
def follow_links_recursive(url: str, depth: int = 4) -> List[str]:
    """Recursively explores hyperlinks on a webpage up to a given depth.

    This tool fetches a webpage and extracts all links from it.
    It then visits each discovered link (if it's a valid URL) and repeats
    the process up to the specified recursion depth.

    Args:
        url: The starting URL to explore.
        depth: The maximum depth of recursion. Depth 0 returns only the original URL.

    Returns:
        A list of all reachable URLs found during the recursive exploration.
        May contain both relative and absolute URLs.
    """
    from urllib.parse import urljoin, urlparse

    visited = set()
    result = set()

    def crawl(current_url, current_depth):
        if current_depth > depth or current_url in visited:
            return
        visited.add(current_url)

        try:
            response = requests.get(current_url, timeout=5)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")
            links = [urljoin(current_url, a["href"]) for a in soup.find_all("a", href=True)]
            for link in links:
                result.add(link)
                crawl(link, current_depth + 1)
        except Exception:
            pass  # Skip errors silently

    crawl(url, 0)
    return list(result)

##########

@tool
def extract_and_classify_links(url: str) -> Dict[str, List[str]]:
    """Extracts all hyperlinks from a webpage and classifies them into categories.

    This tool visits the given URL, extracts all <a href> links, and organizes them into
    the following categories:
    - 'webpages': regular HTML pages or links without extensions
    - 'files': downloadable documents (PDF, CSV, JSON, XML, ZIP, etc.)
    - 'media': images, audio, video files (JPG, MP4, MP3, etc.)
    - 'others': any remaining links

    Args:
        url: The URL of the webpage to scan.

    Returns:
        A dictionary with link categories as keys and lists of corresponding URLs as values.
    """
    file_exts = ('.pdf', '.csv', '.xlsx', '.xls', '.json', '.xml', '.zip')
    media_exts = ('.png', '.jpg', '.jpeg', '.gif', '.mp4', '.mp3', '.wav', '.webm')

    classified = {
        "webpages": [],
        "files": [],
        "media": [],
        "others": [],
    }

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        for a in soup.find_all("a", href=True):
            href = a["href"]
            full_url = urljoin(url, href)
            lower = full_url.lower()

            if any(lower.endswith(ext) for ext in file_exts):
                classified["files"].append(full_url)
            elif any(lower.endswith(ext) for ext in media_exts):
                classified["media"].append(full_url)
            elif lower.startswith("http") and (lower.endswith("/") or "." not in lower.split("/")[-1]):
                classified["webpages"].append(full_url)
            else:
                classified["others"].append(full_url)

        return classified

    except Exception as e:
        return {"error": [f"Error during extraction: {str(e)}"]}

##########

@tool
def get_keyword_context(url: str, keyword: str, window: int = 50) -> List[str]:
    """Extracts excerpts of text around a given keyword from a webpage.

    This tool fetches the content of a webpage, converts it into plain text,
    and searches for occurrences of the keyword. For each occurrence, it returns
    a snippet that includes `window` words before and after the keyword.

    Args:
        url: The URL of the webpage to analyze.
        keyword: The keyword to search for in the text (case-insensitive).
        window: The number of words to include before and after each match.

    Returns:
        A list of contextual excerpts where the keyword appears, or an error message if the request fails.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator=' ', strip=True)

        words = text.split()
        keyword_lower = keyword.lower()
        contexts = []

        for i, word in enumerate(words):
            if keyword_lower in word.lower():
                start = max(i - window, 0)
                end = min(i + window + 1, len(words))
                context = ' '.join(words[start:end])
                contexts.append(context)

        return contexts if contexts else [f"No occurrences of '{keyword}' found."]

    except Exception as e:
        return [f"Error while extracting context: {str(e)}"]

### Data Agent tools

In [14]:
@tool
def find_relevant_documents(query: str) -> List[str]:
    """Finds the most relevant documents for a given user query using the local RAG system.

    This tool queries the vector and sparse retrievers to find the most contextually relevant
    documents or file paths related to the input query.

    Args:
        query: The user's question or search string.

    Returns:
        A list of strings representing the most relevant documents or paths.
    """
    sources = rag.find_relevant_documents(query)
    return sources

##########

def parse_pdf(path: str) -> str:
    import fitz  # PyMuPDF
    try:
        doc = fitz.open(path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    except Exception as e:
        return f"Error parsing PDF: {str(e)}"

def parse_csv(path: str) -> str:
    import pandas as pd
    try:
        df = pd.read_csv(path)
        return df.head(10).to_markdown()
    except Exception as e:
        return f"Error parsing CSV: {str(e)}"
    
def parse_json(path: str) -> str:
    import json
    try:
        with open(path, "r") as f:
            data = json.load(f)
        return json.dumps(data, indent=2)[:3000]  # Trimmed for safety
    except Exception as e:
        return f"Error parsing JSON: {str(e)}"

def parse_xml(path: str) -> str:
    import xml.etree.ElementTree as ET
    try:
        tree = ET.parse(path)
        root = tree.getroot()

        def parse_element(elem, level=0):
            text = f"{'  ' * level}<{elem.tag}>: {elem.text.strip() if elem.text else ''}\n"
            for child in elem:
                text += parse_element(child, level + 1)
            return text

        return parse_element(root)
    except Exception as e:
        return f"Error parsing XML: {str(e)}"
     
###########

@tool
def detect_and_parse(path: str) -> str:
    """Automatically detects the type of a local file and parses its content accordingly.

    This tool acts as a smart wrapper that routes the file to the appropriate parser based
    on its extension (PDF, CSV, JSON, XML). It is ideal for agents that don't know in advance
    what type of file they are dealing with.

    Use this tool when the user provides a file path and wants to:
    - View or analyze the content, regardless of the file type.
    - Extract text or structure without needing to specify the format.

    Args:
        path: Path to the file.

    Returns:
        The parsed content or structure of the file, or an error message.
    """
    ext = path.lower().split(".")[-1]
    if ext == "pdf":
        return parse_pdf(path)
    elif ext == "csv":
        return parse_csv(path)
    elif ext == "json":
        return parse_json(path)
    elif ext == "xml":
        return parse_xml(path)
    else:
        return "Unsupported file type"

###########

@tool
def get_document_metadata(path: str) -> Dict[str, str]:
    """Returns metadata information about a local document file.

    Use this tool when the user wants to inspect technical information about a file such as
    size, type, name, and last modification date.

    Args:
        path: The local path to the document file.

    Returns:
        A dictionary containing metadata about the file.
    """
    try:
        stat = os.stat(path)
        metadata = {
            "name": os.path.basename(path),
            "size (KB)": f"{stat.st_size // 1024}",
            "last_modified": datetime.fromtimestamp(stat.st_mtime).isoformat(),
            "type": os.path.splitext(path)[1].lower()
        }
        return metadata
    except Exception as e:
        return {"error": str(e)}

###########    
    
@tool
def get_keyword_context(path: str, keyword: str, window: int = 50) -> List[str]:
    """Extracts short text segments around a keyword from a local document.

    Use this tool when the user is searching for how a specific term is used inside a
    document. It returns snippets that include the keyword and surrounding words.

    Args:
        path: Path to the local file.
        keyword: The target word or phrase to search.
        window: Number of words before and after the keyword to include.

    Returns:
        A list of text excerpts, or a message if no match is found.
    """
    try:
        content = detect_and_parse(path)
        words = content.split()
        keyword_lower = keyword.lower()
        contexts = []

        for i, word in enumerate(words):
            if keyword_lower in word.lower():
                start = max(i - window, 0)
                end = min(i + window + 1, len(words))
                context = ' '.join(words[start:end])
                contexts.append(context)

        return contexts if contexts else [f"No occurrences of '{keyword}' found."]

    except Exception as e:
        return [f"Error while extracting context: {str(e)}"]

### Définition des Agents

In [ ]:
rag_agent = ToolCallingAgent(
    tools = [agent_add_file_to_rag, extract_any_archive, move_file, normalize_and_ensure_unique_filename],
    model = model,
    add_base_tools = True,
    max_steps = 10,
    name = "rag_agent",
    description = """
rag_agent is responsible for managing the preparation and integration of all external files into the local Retrieval-Augmented Generation (RAG) system.
It monitors the 'Download' folder, where web_agent stores newly retrieved files, and performs a full processing pipeline for each item.

Its tasks include: detecting archive files (e.g., ZIP), extracting their contents, normalizing filenames to prevent conflicts, and moving valid files
into the dedicated 'Data' folder where the RAG operates. Once a file is in place, rag_agent indexes it by chunking and storing it in the RAG system
so that data_agent can later retrieve relevant content from it.

rag_agent ensures all downloaded resources are cleanly integrated, deduplicated, and properly stored, serving as the entry point for new knowledge
in the local ecosystem. It plays a critical role in keeping the knowledge base up to date and consistent for future querying.
"""
    )

#############################################################

data_agent = ToolCallingAgent(
    tools = [find_relevant_documents, detect_and_parse, get_document_metadata, get_keyword_context],
    model = model,
    add_base_tools = True,
    max_steps = 10,
    name = "data_agent",
    description ="""
This agent is called data_agent and must be call before the other agent to find local files to answer to the query.
data_agent is a local data analysis agent responsible for retrieving and extracting relevant information from documents stored on disk.
It works with a local Retrieval-Augmented Generation (RAG) system to identify the most contextually relevant document chunks for a given query.
However, it must not rely blindly on the RAG: it critically inspects each result by parsing the file contents and validating their relevance.

data_agent supports multiple file types, including PDF, CSV, JSON, and XML. It can extract passages around keywords, summarize documents,
or return detailed metadata depending on the query. It is also capable of navigating the local folder structure manually,
opening and inspecting files that were not returned by the RAG, when necessary.

Its primary objective is to assist manager_agent by providing clear, structured, and well-justified pieces of local data
to support complex queries, always ensuring accuracy, relevance, and source traceability.
"""
    )

#############################################################

web_agent = ToolCallingAgent(
    tools = [visit_webpage, summarize_webpage, check_url_validity, download_file, follow_links_recursive, extract_and_classify_links, GoogleSearchTool()],
    model = model,
    add_base_tools = True,
    max_steps = 20,
    name="web_agent",
    description ="""
This agent is called web_agent and must be call only is data_aganet can't provide you all the information to answer to the query.
web_agent is a specialized autonomous web exploration agent designed to retrieve high-quality, factual information from the internet to support queries from manager_agent.
Its mission is to locate and extract relevant official documents and datasets that can help answer questions related to the French National Assembly.

Its absolute and strict priority is to focus on the two official websites of the Assembly:
- https://www.assemblee-nationale.fr/
- https://data.assemblee-nationale.fr/

web_agent must begin by exploring these two sites recursively, using tools such as extract_and_classify_links and follow_links_recursive
to discover all relevant resources. It should manually navigate from one link to another, avoiding any generated queries or synthetic links.

Only if no relevant content can be retrieved after a complete recursive crawl of both sites is web_agent authorized to use the GoogleSearchTool.
Even then, it must carefully extract real URLs from search results and explore them manually, without generating new links.

When useful files (e.g., PDFs, CSVs, JSONs, XMLs) are found and downloaded, web_agent should return them to manager_agent so that they can be
indexed by the RAG system. Its purpose is to act as a precise, reliable, and traceable information gatherer focused on official sources first.
""")

#############################################################

manager_agent = CodeAgent(
    tools = [],
    model = model,
    managed_agents = [web_agent, data_agent, rag_agent],
    additional_authorized_imports = ["time", "numpy", "pandas"],
    planning_interval = 3,
    verbosity_level = 2,
    #add_base_tools = True
    max_steps = 10,
    )


In [31]:
general_manager_agent = CodeAgent(
    tools = [],
    model = model,
    managed_agents = [web_agent, data_agent, rag_agent],
    additional_authorized_imports = ["time", "numpy", "pandas"],
    planning_interval = 3,
    verbosity_level = 2,
    #add_base_tools = True
    max_steps = 10,
    )

general_agent = ToolCallingAgent(
    tools = [find_relevant_documents, detect_and_parse, get_document_metadata, get_keyword_context],
    model = model,
    add_base_tools = True,
    max_steps = 10,
    name = "data_agent",
    description ="")

In [50]:
manager_agent_prompt = """

#############################################################

You are manager_agent, the coordinator and reasoning agent responsible for answering complex and specific questions
related to the French National Assembly (Assemblée nationale). You do not search or parse documents yourself — instead,
you intelligently orchestrate the work of the other agents to build a structured, factual, and well-sourced answer.

AGENTS AT YOUR DISPOSAL:

1. data_agent — analyzes local files using a RAG system and manual inspection. It retrieves the most relevant documents
   from the local filesystem and extracts key information (PDF, CSV, JSON, XML). It provides factual, structured content for synthesis.

2. web_agent — explores the web to retrieve official documents. It begins with the two Assembly websites
   (assemblee-nationale.fr and data.assemblee-nationale.fr), navigates them recursively, and only uses broader web search
   if no results are found locally. It returns raw files or web content for indexing.

3. rag_agent — manages ingestion and indexing of files into the RAG system. When a new file is downloaded, you must call rag_agent
   to normalize, move, extract, chunk, and add the file to the retrieval system.

YOUR STRATEGY (VERY IMPORTANT):

1. ALAWAYS Start with local knowledge using data_agent:
   - Always begin by calling data_agent to retrieve relevant documents from the local RAG.
   - Evaluate whether the returned content is sufficient to answer the question.

2. Only if local data is insufficient go search on web:
   - Call web_agent to search for complementary or missing information online.
   - Instruct web_agent to search only the official Assembly websites first, then escalate to broader web search only if needed.

3. When new files are downloaded:
   - You must ask rag_agent to process them so they are added to the RAG system.
   - Only after ingestion should you re-query data_agent with the original question.

4. Final step — synthesis:
   - Once all relevant data is collected, write a structured, concise, and well-sourced answer.
   - Prefer factual statements, short summaries, and clear source attribution (file names, dates, links).
   - Clearly mention whether the information came from local data or external sources.

GUIDELINES:

- Be transparent in your reasoning: keep track of all the steps taken.
- Do not skip ahead — always try to solve the query with local data first.
- Avoid hallucinating facts. Always rely on data returned by other agents.
- When no answer is found, clearly explain the limitation and suggest reformulating the query.

Your goal is to produce a trustworthy, well-organized response that is explainable and verifiable, based on a chain of carefully delegated tasks and critical synthesis.
"""

#############################################################

web_agent_prompt = """

#############################################################

You are web_agent, a focused web exploration assistant working under the supervision of manager_agent.
Your primary mission is to locate, extract, and return precise and reliable documents or datasets from the web
to support complex questions about the French National Assembly.

IMPORTANT STRATEGY:

1. Your exploration must ALWAYS start with the two official Assembly websites:
   - https://www.assemblee-nationale.fr/
   - https://data.assemblee-nationale.fr/

These are your highest priority sources. You must exhaust them before exploring anywhere else.

2. Use recursive navigation:
   - Begin from the main page of each site.
   - Use the tools extract_and_classify_links and follow_links_recursive to explore pages.
   - Follow internal links manually, one by one, as they appear on the site.
   - Avoid generating search terms or skipping directly to other URLs — your exploration must simulate real user navigation.
   - You are allowed to dig as deep as necessary using recursive link exploration up to 4 or 5 levels deep.

3. Use keywords to guide navigation:
   - Based on the question from manager_agent, identify important keywords (e.g. deputy, list, mandate, vote, legislature).
   - Prioritize links that contain these keywords or are related to them.
   - Extract contexts, summaries, and datasets connected to these concepts.

4. If you discover downloadable files (PDFs, CSVs, JSON, XML):
   - If you can open it as a CSV file, you can do it and extract information dirrectily 
   - if you can't extract informations, Download the file using download_file.
   - Return the file to manager_agent so that it can be stored and indexed via rag_agent.
   - Do not attempt to interpret the content of the files yourself.

5. Only if no useful information is found after a deep and methodical exploration of both official sites:
   - You are then authorized to use the GoogleSearchTool with precise keywords.
   - From the search results, extract the resulting links.
   - As before, visit these links one by one manually and use recursive browsing tools to explore.

RULES AND MINDSET:

- Do not generate or invent links or content.
- Always prioritize official sources and institutional data over general internet content.
- Avoid blogs, opinion articles, or third-party aggregators unless absolutely necessary.
- Focus on structure, traceability, and reliability.
- Be rigorous and patient: exhaustive navigation is required before escalating to external search.

Your job is not to summarize or analyze, but to return well-targeted content to your manager_agent, ideally in the form of datasets or structured documents ready for ingestion into a local RAG system.
"""

#############################################################

data_agent_prompt = """

#############################################################

You are data_agent, a local document analysis assistant working under the supervision of manager_agent.
Your role is to examine documents stored on the local file system in order to extract structured and relevant data
that can help answer complex questions related to the French National Assembly.

HOW TO OPERATE:

1. Begin by querying the local RAG system:
   - Use the find_relevant_documents tool to retrieve document chunks related to the question.
   - The RAG system uses keyword-based retrieval (BM25) to identify relevant content.

2. Maintain critical judgment:
   - Do not rely solely on the RAG output.
   - For each suggested document, use detect_and_parse to read and inspect the actual content.
   - Use get_keyword_context to extract passages around important terms.
   - Use get_document_metadata to evaluate file relevance (modification date, type, size).

3. Think beyond the RAG:
   - If the RAG does not return coherent results, you are allowed to manually explore the local folder structure.
   - You can inspect filenames, open files directly, and analyze their content for relevance — even if they were not returned by the RAG.
   - Follow common-sense logic to locate documents: e.g., files that contain “deputies”, “mandates”, “elections”, “votes” are likely to be useful.

4. Data extraction:
   - You are expected to extract only factual, relevant, and structured information.
   - You can combine results from multiple files or sections if needed.
   - You are free to use all file types: PDF, CSV, JSON, XML, or others.

5. Output:
   - Your goal is to return to manager_agent the most accurate and informative local data possible.
   - Do not generate answers — just surface the best material to support the final synthesis.

PRINCIPLES:

- Precision over quantity: do not return irrelevant content.
- Structure over narrative: focus on tables, figures, lists, named entities, and dates.
- RAG is helpful, but your human-like judgment is just as important.
- You are the last line of quality control before content goes into the final answer.

Your job is to make sure that every local document potentially useful to the question is inspected, parsed, and delivered in a reliable form to manager_agent.
"""

#############################################################

rag_agent_prompt = """

#############################################################

You are rag_agent, a system assistant responsible for preparing and indexing new files into the local Retrieval-Augmented Generation (RAG) system.
You are called by manager_agent whenever new files have been downloaded by web_agent and need to be processed and added to the knowledge base.

You operate by systematically inspecting and handling every file located in the 'Download' directory. Your procedure is as follows:

1. Inspect the Downloads folder:
   - List all files present in the Downloads folder : /users/formation/irtn7prtnc/llm_engineering/Downloads directory.
   - For each file, determine its type (e.g., ZIP archive, PDF, CSV, JSON, XML).

2. If the file is an archive (ZIP or similar):
   - Extract its content using the appropriate extraction tool.
   - Treat all extracted files as if they were directly downloaded.
   - Once extracted, delete the original archive to avoid duplication.

3. Normalize and verify each file:
   - Use the normalize_and_ensure_unique_filename function to make sure there are no naming conflicts.
   - If necessary, append a suffix to avoid overwriting an existing file.

4. Move each file to the 'Data' directory "/users/formation/irtn7prtnc/llm_engineering/Data":
   - Use the move_file function to relocate the file from 'Download' to 'Data'.
   - Ensure the final destination path is clean and traceable.

5. Add the file to the RAG system:
   - Use the agent_add_file_to_rag function to chunk and index the file into the RAG database.
   - This step is mandatory for each successfully handled document.

6. Report back to manager_agent:
   - For each file, confirm whether it was successfully processed, moved, and indexed.
   - If an error occurred, report it clearly with the filename and reason.

RULES:

- Never ignore or skip a file unless it is unsupported.
- Always clean up intermediate files (archives) once extracted.
- Never modify file content — only prepare and route them correctly.
- Your job is purely organizational and preparatory — you do not interpret or use the content.

Your purpose is to ensure that all external files retrieved by web_agent are cleanly and reliably integrated into the local RAG system, ready for future use by data_agent.
"""


In [51]:
manager_agent.prompt_templates["system_prompt"] = general_manager_agent.prompt_templates["system_prompt"] + manager_agent_prompt
web_agent.prompt_templates["system_prompt"] = general_agent.prompt_templates["system_prompt"] + web_agent_prompt
data_agent.prompt_templates["system_prompt"] = general_agent.prompt_templates["system_prompt"] + data_agent_prompt
rag_agent.prompt_templates["system_prompt"] = general_agent.prompt_templates["system_prompt"] + rag_agent_prompt

NameError: name 'general_manager_agent' is not defined

In [66]:
print(manager_agent.prompt_templates["system_prompt"])

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

# Utilisation de l'agent

In [68]:
agent_output = manager_agent.run('Give me the exact date of first and  the last time Jean Lassale was ellected as french depute')

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the exact date of first and  the last time Jean Lassale was ellected as french depute                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5-coder:32b ───────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. **Task Assignment to web_agent**:
   - Request: Explore the official websites of the French National Assembly (https://www.assemblee-nationale.fr/ 
and https://data.assemblee-nationale.fr/) recursively to find all documents related to Jean Lassale's election 
history as a French depute.

2. **Task Assignment to rag_agent**:
   - Request: Process any downloaded files from web_agent, ensure they are indexed correctly in the RAG system for 
data retrieval by data_agent.

3. **Task Assignment to data_agent**:
   - Request: Retrieve and analyze documents stored locally in the Data folder related to Jean Lassale's election 
history. Extract specific information regarding his first and last elections as a French depute.

4. **Compile Results**:
   - Compile the extracted dates from the analyzed documents into a coherent format.

5. **Final Answer Generation**:
   - Generate the final answer containing the exact date of Jean Lassale's first and last election as a French 
depute.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will start by using data_agent to retrieve any relevant local documents related to Jean Lassale's       
election history. If the local data is insufficient, I will then use web_agent to search for more information on   
the official websites of the French National Assembly.                                                             
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# First, query local data using data_agent                                                                         
jean_lassale_elections = data_agent(query="election history Jean Lassale")                                         
print(jean_lassale_elections)                                                                                      
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # First, query local data using data_agent                                                                       
  jean_lassale_elections = data_agent(query="election history Jean Lassale")                                       
  print(jean_lassale_elections)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: It appears that the `final_answer` function is being called with an incorrect argument. The error         
indicates that `final_answer` does not accept a keyword argument named 'answer'. I will correct this by passing the
argument directly without using the keyword.                                                                       
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# Correctly calling final_answer to provide the final result                                                       
final_answer("First election date: June 8, 1997\nLast election date: February 14, 2022")                           
```                                                                                                                
Calling tools:                                                                                                     
[{'id': 'call_11', 'type': 'function', 'function': {'name': 'python_interpreter', 'arguments': '# Correctly calling
final_answer to provide the final result\nfinal_answer("First election date: June 8, 1997\\nLast election date:    
February 14, 2022")'}}]Calling tools:                                                                              
[{'id': 'call_11', 'type': 'function', 'function': {'name': 'python_interpreter', 'arguments': '# Correctly calling
final_answer to provide the final result\nfinal_answer("First election date: June 8, 1997\\nLast election date:    
February 14, 2022")'}}]                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Correctly calling final_answer to provide the final result                                                     
  final_answer("First election date: June 8, 1997\nLast election date: February 14, 2022")                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: First election date: June 8, 1997
Last election date: February 14, 2022

[Step 7: Duration 9.44 seconds| Input tokens: 80,170 | Output tokens: 2,069]

Final output:
First election date: June 8, 1997
Last election date: February 14, 2022
